In [1]:
import numpy as np
import sympy as sym

In [2]:
x = sym.Symbol('x',real=True)
y = sym.Symbol('y',real=True)

In [3]:
def GetHermite(n,x):

    if n==0:
        poly = sym.Number(1)
    elif n==1:
        poly = 2*x
    else:
        poly = (2*x*GetHermite(n-1,x))-(2)*(n-1)*GetHermite(n-2,x)
   
    return sym.expand(poly,x)

In [4]:
GetHermite(3,x)

8*x**3 - 12*x

In [5]:
def GetDHermite(n,x):
    Pn = GetHermite(n,x)
    return sym.diff(Pn,x,1)

In [6]:
def GetNewton(f,df,xn,itmax=10000,precision=1e-14):
    
    error = 1.
    it = 0
    
    while error >= precision and it < itmax:
        
        try:
            
            xn1 = xn - f(xn)/df(xn)
            
            error = np.abs(f(xn)/df(xn))
            
        except ZeroDivisionError:
            print('Zero Division')
            
        xn = xn1
        it += 1
        
    if it == itmax:
        return False
    else:
        return xn

In [7]:
def GetRoots(f,df,x,tolerancia = 10):
    
    Roots = np.array([])
    
    for i in x:
        
        root = GetNewton(f,df,i)

        if  type(root)!=bool:
            croot = np.round( root, tolerancia )
            
            if croot not in Roots:
                Roots = np.append(Roots, croot)
                
    Roots.sort()
    
    return Roots

In [8]:
def GetAllRootsGHer(n):

    xn = np.linspace(-np.sqrt(4+n+1),np.sqrt(4+n+1),100)
    
    Hermite = []
    DHermite = []
    
    for i in range(n+1):
        Hermite.append(GetHermite(i,x))
        DHermite.append(GetDHermite(i,x))
    
    poly = sym.lambdify([x],Hermite[n],'numpy')
    Dpoly = sym.lambdify([x],DHermite[n],'numpy')
    Roots = GetRoots(poly,Dpoly,xn)

    if len(Roots) != n:
        ValueError('El número de raíces debe ser igual al n del polinomio.')
    
    return Roots

In [9]:
GetAllRootsGHer(3)

array([-1.22474487,  0.        ,  1.22474487])

In [10]:
def GetWeightsGHer(n):

    Roots = GetAllRootsGHer(n)

    Hermite = []
    
    for i in range(n):
        Hermite.append(GetHermite(i,x))
        
    
    poly = sym.lambdify([x],Hermite[n-1],'numpy')
    Weights = (2**(n-1))*(np.math.factorial(n))*(np.sqrt(np.pi))/((n**2)*(poly(Roots))**2)
    
    return Weights

In [11]:
GetWeightsGHer(3)

array([0.29540898, 1.1816359 , 0.29540898])